# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = "output_data/cities_elh.csv"
vacation_df = pd.read_csv(filepath)
vacation_df.dropna(inplace=True)
vacation_df.head()


,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind_Speed,date
0,Lagoa,PT,39.0500,-27.9833,20.37,75,100,7.54,1630898843
1,Qaanaaq,GL,77.4840,-69.3632,-7.47,86,98,0.94,1630898844
2,Nome,US,64.5011,-165.4064,8.04,87,90,3.09,1630898844
3,Faya,SA,18.3851,42.4509,15.29,47,9,0.96,1630898844
4,Albany,US,42.6001,-73.9662,17.95,93,100,1.34,1630898681


In [3]:
vacation_df.describe()

,Lat,Lng,Temperature,Humidity,Clouds,Wind_Speed,date
count,566.000000,566.000000,566.000000,566.000000,566.000000,566.000000,5.660000e+02
mean,19.333340,19.649231,19.314152,70.908127,54.086572,3.423569,1.630899e+09
std,32.763769,91.267482,7.846022,21.272577,40.241500,2.450866,9.087011e+01
min,-54.800000,-179.166700,-7.470000,10.000000,0.000000,0.000000,1.630898e+09
25%,-6.217125,-62.236300,13.817500,59.250000,7.250000,1.540000,1.630899e+09
50%,21.315100,21.047650,20.185000,76.000000,64.000000,2.930000,1.630899e+09
75%,45.689825,103.412525,25.555000,87.000000,97.000000,4.867500,1.630899e+09
max,78.218600,179.316700,35.780000,100.000000,100.000000,13.420000,1.630899e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = vacation_df[['Lat', 'Lng']]
weights = vacation_df['Humidity']
figure_layout = {'width':'800px', 'height':'400px', 'border':'1px solid blue', 'padding':'0px'}
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=7, dissipating=False))
fig

Figure(layout=FigureLayout(border='1px solid blue', height='400px', padding='0px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_conditions = vacation_df[(vacation_df['Temperature'] >= 26) & (vacation_df['Temperature'] <= 35)]
vacation_conditions = vacation_conditions[vacation_conditions['Wind_Speed'] <= 10]
vacation_conditions = vacation_conditions[vacation_conditions['Clouds'] <= 10]
vacation_conditions = vacation_conditions[vacation_conditions['Humidity'] <= 60]
vacation_conditions.describe()

,Lat,Lng,Temperature,Humidity,Clouds,Wind_Speed,date
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,1.300000e+01
mean,29.451062,16.279685,29.281538,20.615385,0.769231,2.537692,1.630899e+09
std,17.265471,80.936404,2.575318,7.455853,1.640825,1.757655,6.166504e+01
min,-24.866700,-121.423800,26.880000,10.000000,0.000000,0.240000,1.630899e+09
25%,29.033100,14.428300,27.530000,15.000000,0.000000,1.790000,1.630899e+09
50%,33.595900,40.917100,28.480000,19.000000,0.000000,2.550000,1.630899e+09
75%,37.104100,56.924400,30.040000,24.000000,1.000000,3.020000,1.630899e+09
max,43.801000,113.633300,34.740000,37.000000,6.000000,7.200000,1.630899e+09


# Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_conditions
hotel_df

,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind_Speed,date
65,Abu Kamal,SY,34.4506,40.9171,27.56,24,0,3.02,1630898866
74,Carnarvon,AU,-24.8667,113.6333,30.04,31,0,7.20,1630898869
165,Dehlorān,IR,32.6941,47.2679,32.07,10,0,3.57,1630898902
200,Sakakah,SA,29.9697,40.2064,27.76,17,6,2.11,1630898917
207,Jalu,LY,29.0331,21.5482,28.48,37,0,3.34,1630898920
227,Twentynine Palms,US,34.1356,-116.0542,34.74,15,0,0.24,1630898927
247,Tabas,IR,33.5959,56.9244,26.88,19,0,2.10,1630898934
255,Laguna,US,38.4210,-121.4238,28.75,26,1,0.45,1630898937
288,Ürümqi,CN,43.8010,87.6005,27.53,18,0,3.00,1630898949
330,Kumul,CN,42.8000,93.4500,27.18,15,1,2.69,1630898966


In [7]:
hotel_df['Hotel_Name'] = ''
hotel_df

,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind_Speed,date,Hotel_Name
65,Abu Kamal,SY,34.4506,40.9171,27.56,24,0,3.02,1630898866,
74,Carnarvon,AU,-24.8667,113.6333,30.04,31,0,7.20,1630898869,
165,Dehlorān,IR,32.6941,47.2679,32.07,10,0,3.57,1630898902,
200,Sakakah,SA,29.9697,40.2064,27.76,17,6,2.11,1630898917,
207,Jalu,LY,29.0331,21.5482,28.48,37,0,3.34,1630898920,
227,Twentynine Palms,US,34.1356,-116.0542,34.74,15,0,0.24,1630898927,
247,Tabas,IR,33.5959,56.9244,26.88,19,0,2.10,1630898934,
255,Laguna,US,38.4210,-121.4238,28.75,26,1,0.45,1630898937,
288,Ürümqi,CN,43.8010,87.6005,27.53,18,0,3.00,1630898949,
330,Kumul,CN,42.8000,93.4500,27.18,15,1,2.69,1630898966,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "lodging",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [12]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel_Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Skipping")
    

    time.sleep(1)

print('---Done---')

Retrieving Results for Index 65: Abu Kamal.
Skipping
Retrieving Results for Index 74: Carnarvon.
Closest hotel in Carnarvon is Hospitality Carnarvon.
Retrieving Results for Index 165: Dehlorān.
Closest hotel in Dehlorān is هتل سعید.
Retrieving Results for Index 200: Sakakah.
Closest hotel in Sakakah is Raoum Inn Hotel.
Retrieving Results for Index 207: Jalu.
Closest hotel in Jalu is Jalu Hotel.
Retrieving Results for Index 227: Twentynine Palms.
Closest hotel in Twentynine Palms is Holiday Inn Express & Suites Twentynine Palms- Joshua Tree, an IHG Hotel.
Retrieving Results for Index 247: Tabas.
Closest hotel in Tabas is هتل.
Retrieving Results for Index 255: Laguna.
Closest hotel in Laguna is Hampton Inn & Suites Sacramento-Elk Grove Laguna I-5.
Retrieving Results for Index 288: Ürümqi.
Closest hotel in Ürümqi is Sheraton Urumqi Hotel.
Retrieving Results for Index 330: Kumul.
Closest hotel in Kumul is 7 Days Inn.
Retrieving Results for Index 341: Sabha.
Closest hotel in Sabha is فندق ا

In [13]:
hotel_df

,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind_Speed,date,Hotel_Name
65,Abu Kamal,SY,34.4506,40.9171,27.56,24,0,3.02,1630898866,
74,Carnarvon,AU,-24.8667,113.6333,30.04,31,0,7.20,1630898869,Hospitality Carnarvon
165,Dehlorān,IR,32.6941,47.2679,32.07,10,0,3.57,1630898902,هتل سعید
200,Sakakah,SA,29.9697,40.2064,27.76,17,6,2.11,1630898917,Raoum Inn Hotel
207,Jalu,LY,29.0331,21.5482,28.48,37,0,3.34,1630898920,Jalu Hotel
227,Twentynine Palms,US,34.1356,-116.0542,34.74,15,0,0.24,1630898927,Holiday Inn Express & Suites Twentynine Palms-...
247,Tabas,IR,33.5959,56.9244,26.88,19,0,2.10,1630898934,هتل
255,Laguna,US,38.4210,-121.4238,28.75,26,1,0.45,1630898937,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
288,Ürümqi,CN,43.8010,87.6005,27.53,18,0,3.00,1630898949,Sheraton Urumqi Hotel
330,Kumul,CN,42.8000,93.4500,27.18,15,1,2.69,1630898966,7 Days Inn


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(border='1px solid blue', height='400px', padding='0px', width='800px'))